<a href="https://colab.research.google.com/github/JaswanthBadvelu/CVRP/blob/main/CVRP_with_saving_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
import operator
from audioop import reverse
from ortools.constraint_solver._pywrapcp import new_BaseLns
from itertools import repeat
import requests
import math

def manhattanDistance( xy1, xy2 ):
    "Returns the Manhattan distance between points xy1 and xy2"
    return abs( xy1[0] - xy2[0] ) + abs( xy1[1] - xy2[1] )

def euclideanDistance( xy1, xy2 ):
    "Returns the Manhattan distance between points xy1 and xy2"
    return  math.sqrt(( xy2[0] - xy1[0] )**2 + ( xy2[1] - xy1[1] )**2)

def calculateRouteCost(r):
    total = 0
    for i in range(len(r) - 1):
        total+= euclideanDistance(r[i] , r[i+1])
    return total

def addDepotAtEnds(depot,route):
    route.insert(0,depot)
    route.append(depot)

url = 'https://raw.githubusercontent.com/JaswanthBadvelu/CVRP/main/data.json'
resp = requests.get(url)
data = json.loads(resp.text)

noOfCustomers = len(data["nodes"])
customerPosDemand = {}
vehicleCap = data["vehicleCapacity"][0]["1"]
#print(vehicleCap)
for i in range(noOfCustomers):
    customerPosDemand[data["nodes"][i]["x"],data["nodes"][i]["y"]] = data["nodes"][i]["demand"]

#compute Savings for depot and i,j where i <> j
def computeSaving(depot, i,j):
    iDepot = manhattanDistance(i, depot)
    jDepot = manhattanDistance(depot, j)
    ijDist = manhattanDistance(i, j)
    
    return (iDepot + jDepot - ijDist)
    
#def distDepot


#calculating savingss for all pairs
savings = {}
customerPositions =  list(customerPosDemand)
pointsLen = len(customerPositions)
depot = (data["depot"]["x"],data["depot"]["y"])


def allCustomersConsidered(customerServed):
    for val in customerServed.values():
        if val == False:
            return False
    return True

#Step 1
distanceDict = {}
for i in range(pointsLen):
    for j in range(i+1,pointsLen):
        distanceDict[(customerPositions[i], customerPositions[j])] = euclideanDistance(customerPositions[i], customerPositions[j])
#print(distanceDict)
#Step 2
for i in range(pointsLen):
    for j in range(i+1,pointsLen):
        savings[customerPositions[i], customerPositions[j]]  =  computeSaving(depot,customerPositions[i], customerPositions[j])
savings = sorted(savings.items(),key=operator.itemgetter(1),reverse=True)

l = len(savings)
cust_pairs = list()
for i in range(l):
    cust_pairs.append(savings[i][0])

#initially none of the customers have been isServed
customerServed = {}
for c in customerPositions:
    customerServed[c] = False
l = len(savings)
cust_pairs = list()
for i in range(l):
    cust_pairs.append(savings[i][0])

#initially none of the customers have been isServed
customerServed = {}
for c in customerPositions:
    customerServed[c] = False
    
#Step 3
def inPrevious(new,existing):
    start = existing[0]
    end = existing[len(existing)-1]
    if new == start:
        return 1
    elif new == end:
        return 0
    else:
        return -1

def capacityValid(existing,new):
    totalCap = customerPosDemand[new]
    for c in existing:
        totalCap+=customerPosDemand[c]

    return totalCap <= vehicleCap

def isServed(c):
    return customerServed[c]

def hasBeenServed(c):
    customerServed[c] = True             
     
routes = {}
l = len(cust_pairs)
i = 0
idx = -1
truck = [0,0,0,0,0]

while (not(allCustomersConsidered(customerServed))):
    #choosing the maximum savings customers who are unserved
    for c in cust_pairs:
        if (isServed(c[0]) == False and isServed(c[1]) == False):
            hasBeenServed(c[0])
            hasBeenServed(c[1])
            idx += 1
            routes[idx] = ([c[0],c[1]]) 
            break
     #finding a cust that is either at the start or end of previous route
    for c in cust_pairs:
        res = inPrevious(c[0], routes[idx])
        if res == 0 and capacityValid(routes[idx], c[1]) and (isServed(c[1]) == False):
            if c[1] == (13,7):
                print(customerServed[c[1]])
            hasBeenServed(c[1])
            routes[idx].append(c[1]) 
        elif res == 1 and capacityValid(routes[idx], c[1]) and (isServed(c[1]) == False):
            if c[1] == (13,7):
                print(customerServed[c[1]])
            hasBeenServed(c[1])
            routes[idx].insert(0,c[1])
        
        else:
            res = inPrevious(c[1], routes[idx])
            if res == 0 and capacityValid(routes[idx], c[0]) and (isServed(c[0]) == False):
                if c[0] == (13,7):
                    print(customerServed[c[0]])
                hasBeenServed(c[0])
                routes[idx].append(c[0]) 
            elif res == 1 and capacityValid(routes[idx], c[0]) and (isServed(c[0]) == False):
                if c[0] == (13,7):
                    print(customerServed[c[0]])
                hasBeenServed(c[0])
                routes[idx].insert(0,c[0])

print('All Customer Points')
for point in customerPosDemand.keys():
    print(point)
    
    

print("The route of the trucks")

for r in routes.values():
    for point in r:
        print(point)
print("--------------")

# printing each truck load
for r in routes.values():
    cap = 0
    for points in r:
        cap += customerPosDemand[points]
    print('Total Capacity used:',cap)

print("--------------")
    
#adding depot at ends
for r in routes.values():
    addDepotAtEnds(depot, r)
totalDist = 0
for k,v in routes.items():
    totalDist += calculateRouteCost(v)
    print( 'Truck:', k+1,"Route Travelled",v)
print('The total distance travelled by Trucks',totalDist)

All Customer Points
(47, 51)
(48, 66)
(63, 14)
(42, 45)
(33, 15)
(45, 52)
(13, 16)
(62, 68)
(12, 16)
(32, 63)
The route of the trucks
(33, 15)
(13, 16)
(12, 16)
(32, 63)
(48, 66)
(62, 68)
(63, 14)
(42, 45)
(45, 52)
(47, 51)
--------------
Total Capacity used: 83
Total Capacity used: 100
Total Capacity used: 89
--------------
Truck: 1 Route Travelled [(50, 50), (33, 15), (13, 16), (12, 16), (50, 50)]
Truck: 2 Route Travelled [(50, 50), (32, 63), (48, 66), (62, 68), (63, 14), (50, 50)]
Truck: 3 Route Travelled [(50, 50), (42, 45), (45, 52), (47, 51), (50, 50)]
The total distance travelled by Trucks 278.28256873728236
